In [1]:
!pip install wandb pandas transformers accelerate trl bitsandbytes peft datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 12.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 19.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 22.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 24.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, random_split
import wandb
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import time
import torch
from torch.utils.data import Dataset

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
df = pd.read_csv("./combined.csv")

In [13]:
import re

def board_to_fen(board, player, move_number):
    # Extract rows and reverse to match FEN order (from 8th to 1st rank)
    rows = board.strip().split('\n')

    # Convert board rows to FEN format
    fen_rows = []
    for row in rows:
        clean_row = re.sub(r"\s+", "", row)  # Remove spaces
        # Correctly replace consecutive dots with the count
        fen_row = re.sub(r"(\.+) ", lambda m: str(len(m.group(1))), clean_row)  # Incorrectly placed space
        fen_row = re.sub(r"\.+", lambda m: str(len(m.group(0))), fen_row)  # Handle all dots
        fen_rows.append(fen_row)

    fen_pieces = "/".join(fen_rows)

    # Active color ('w' for White, 'b' for Black)
    active_color = 'w' if player == 'White' else 'b'

    # Default values for other FEN components (assuming basic capabilities)
    castling_availability = 'KQkq'  # Assuming all castling is still possible
    en_passant_target = '-'  # No en passant target
    halfmove_clock = '0'  # Reset on pawn moves or captures, not shown here
    fullmove_number = str(move_number)

    # Compile full FEN string
    fen = f"{fen_pieces} {active_color} {castling_availability} {en_passant_target} {halfmove_clock} {fullmove_number}"
    return fen

df['FEN'] = df.apply(lambda row: board_to_fen(row['Board'], row['Player'], row['Move Number']), axis=1)
df.head()


,Game Number,Move Number,Player,Move,Board,Comment,FEN
0,2263,1,White,d2d4,r n b q k b n r\np p p p p p p p\n. . . . . . ...,NaN,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...
1,2263,2,Black,g8f6,r n b q k b n r\np p p p p p p p\n. . . . . . ...,NaN,rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...
2,2263,3,White,c2c4,r n b q k b . r\np p p p p p p p\n. . . . . n ...,NaN,rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBN...
3,2263,4,Black,g7g6,r n b q k b . r\np p p p p p p p\n. . . . . n ...,NaN,rnbqkb1r/pppppppp/5n2/8/2PP4/8/PP2PPPP/RNBQKBN...
4,2263,5,White,b1c3,r n b q k b . r\np p p p p p . p\n. . . . . n ...,NaN,rnbqkb1r/pppppp1p/5np1/8/2PP4/8/PP2PPPP/RNBQKB...


In [7]:
config = {
    'Model': 'llm',
    'NUMER_OF_DATA_DIRS': 12,
    'batch_size': 4,
    'lr':  1e-4,
    'train_precentege': 0.9,
    'epochs': 2,
    'data_to_use': {'<fen>': True, '<moves>': True, '<last move description>': False,
                    '<legal moves>': False, '<attacked by>': False, '<attacks>': False},
    'max_length':350
}

In [9]:

import json

with open('moves.json', 'r') as f:
  uci_moves = json.load(f)


In [10]:
mvs = list(uci_moves.keys())

In [14]:

def g(df):
    df['historical_moves'] = ""  # Initialize with empty strings
    for index, row in df.iterrows():
        df.loc[index, 'historical_moves'] = ','.join(df.loc[:index, 'Move'][df['Game Number'] == row['Game Number']][-10:])
    return df

df = g(df.copy())
df = df.dropna().reset_index()
print(df)

def convert_data_to_text(row, max_length=config['max_length'], end_of_text_token=""):
    FEN, moves, comment = row['FEN'], row['historical_moves'], row['Comment']
    FEN, moves, comment = FEN[:max_length], moves[:max_length], comment[:max_length]


    token_to_data = {'<fen>': FEN, '<moves>': moves}
    text = ""
    for token, data in token_to_data.items():
        text += f"{token} {data} "
    text += f"<comment> {comment} {end_of_text_token}"
    return text



        index  Game Number  Move Number Player  Move  \
0          22         2263           23  White  e1d2   
1          24         2263           25  White  d4d5   
2          25         2263           26  Black  b8c6   
3          28         2263           29  White  d5d6   
4          29         2263           30  Black  b7b6   
...       ...          ...          ...    ...   ...   
20768  184912            3           82  Black  d6f7   
20769  184915            3           85  White  f3e5   
20770  184920            3           90  Black  f7g5   
20771  184925            3           95  White  b1b6   
20772  184928            3           98  Black  h3c3   

                                                   Board  \
0      r n b . . r k .\np p . . p p b p\n. . . . . . ...   
1      r n b r . . k .\np p . . p p b p\n. . . . . . ...   
2      r n b r . . k .\np p . . p p b p\n. . . . . . ...   
3      r . b r . . k .\np p . . . p b p\n. . n . p . ...   
4      r . b r . . k .\np p

In [15]:
import pickle

sys_prompt = """ You are a professional chess player commentating a game of chess being played. Given below is the board state in FEN notation and previous moves in standard UCI Notation separated by commas. Moves are in order, so last move is the most recent move. Provide some informative commentary about the current move and provide some insights into the play being made.
""".strip()
def gen_prompt(fen,moves,comment):
    return f"""
    ### Instruction: {sys_prompt}

    ### Input :
    FEN : {fen}
    Moves : {moves}

    ### Response :
    {comment}
    <|endoftext|>
    """.strip()


class ProcessDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=config['max_length']):
        self.tokenizer = tokenizer
        self.proccessed_data = []
        self.fen = []
        self.moves = []


        for _, row in df.iterrows():
            fen = row["FEN"]
            h_moves = row["historical_moves"]
            comment = row["Comment"]
            prompt = gen_prompt(fen,h_moves,comment)
            inputs = self.tokenizer(prompt)["input_ids"]
            self.proccessed_data.append(torch.tensor(inputs))
            self.fen.append(fen)
            self.moves.append(h_moves)

    def __len__(self):
        return len(self.proccessed_data)

    def __getitem__(self, index):
        return self.proccessed_data[index]





In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM , BitsAndBytesConfig
from huggingface_hub import PyTorchModelHubMixin
device = "cuda" if torch.cuda.is_available() else "cpu"

#model_type = "google/gemma-7b"  # Or choose another variant
# model_type = "meta-llama/Meta-Llama-3-8B"  # Or choose another variant
model_type = "meta-llama/Meta-Llama-3-8B-Instruct"  # Or choose another variant
#model_type = "meta-llama/Llama-2-7b-hf"  # Pending approval

# model_type = "google/gemma-1.1-2b-it"  # Or choose another variant

access_token = 'your_token_here'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

class LLM(PyTorchModelHubMixin):
    def __init__(self):
        print("Intialization \n")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained(model_type, token = access_token)
        special_tokens_dict = {
            'pad_token': '[PAD]',
            # 'additional_special_tokens': dataset_tokens
        }
        self.tokenizer.add_special_tokens(special_tokens_dict)
        self.tokenizer.add_tokens(mvs)
        self.model = AutoModelForCausalLM.from_pretrained(model_type,quantization_config=bnb_config, token = access_token, force_download=True)
        #self.model = AutoModelForCausalLM.from_pretrained(model_type, token = access_token)
        self.model.resize_token_embeddings(len(self.tokenizer))

    def load_model(self, path):
        print("Loading model\n")
        state_dict = torch.load(path, map_location=self.device)
        self.model.load_state_dict(state_dict,strict=False)
        self.model.to(self.device)

In [19]:
llm = LLM()
model = llm.model
# model.load_state_dict(torch.load("/workspace/models/final.bin"))
model.train()
tokenizer = llm.tokenizer
max_length = config['max_length']
eof = '<|endoftext|>'

Intialization 



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [20]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(130228, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [21]:
games_data_path = '/workspace/ChessCommentary'
saved_models_path = '/workspace/ChessCommentary/models/'

In [22]:
df["Comment"].isna().sum()

0

In [23]:
dataset = ProcessDataset(df,tokenizer)


In [24]:
tokenizer.decode(dataset[0])

"<|begin_of_text|>### Instruction: You are a professional chess player commentating a game of chess being played. Given below is the board state in FEN notation and previous moves in standard UCI Notation separated by commas. Moves are in order, so last move is the most recent move. Provide some informative commentary about the current move and provide some insights into the play being made.\n\n    ### Input :\n    FEN : rnb2rk1/pp2ppbp/6p1/8/3PP3/4BN2/P2q1PPP/1R2KB1R w KQkq - 0 23\n    Moves : f8g7,g1f3,e8g8,c1e3,d8a5,d1d2,c5d4,c3d4,a5d2,e1d2\n\n    ### Response :\n    (In this line White has to accept some risks. The\nking can be an attacking piece (Steinitz). The idea is to leave the King on\nthe Queen side to help in the control of Black's Pawns.)\n    <|endoftext|>"

In [25]:
train_size = int(config['train_precentege'] * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)


In [26]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj","embed_tokens"],
    task_type="CAUSAL_LM",
)

In [27]:
from huggingface_hub import notebook_login
notebook_login()


In [29]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    packing = False,
    dataset_text_field = "proccessed_data",
    args=transformers.TrainingArguments(
        per_device_train_batch_size=config['batch_size'],
        gradient_accumulation_steps=1,
        warmup_steps=2,
        num_train_epochs = 2,
        learning_rate=1e-4,
        logging_steps=100,
        output_dir="outputs_instruct_1e-4",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss


In [ ]:
#validation

In [43]:

pad_token_id = tokenizer('[PAD]')['input_ids'][0]


def generate_commentary(inp):
    inp = inp.unsqueeze(0).to(device)
    output_sequences = model.generate(input_ids=inp,no_repeat_ngram_size=2, max_length=max_length+1, pad_token_id=pad_token_id)
    comment = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return comment

In [45]:
import re
cnt = 0
for data in test_dataset:
    txt = tokenizer.decode(data)
    inp = txt.split("### Response :")[0] + "### Response"
    inp = tokenizer(inp)["input_ids"]
    comment = generate_commentary(torch.tensor(inp))
    print(comment)
    cnt += 1
    if cnt == 5:
      break

### Instruction: You are a professional chess player commentating a game of chess being played. Given below is the board state in FEN notation and previous moves in UCI Notation. Provide some informative commentary about the current move and provide some insights into the play being made.

    ### Input : 
    FEN : 4N1nk/1p1r2pp/p7/2pP1R2/2P5/1P6/P6P/6K1 w KQkq - 0 73 
    Moves : f3e4,f5f4,g4f6,e6f5,e4f5,f4f5,d8d7,c7e8,f6g8,f5f8

    ### Response:
    The game is heating up! With the white pawn on e4, it's clear that the player is going for a strong attack. The pawn is supporting the bishop on c4 and putting pressure on the black position. 

    A clever move, indeed. It's not just about pushing the pawn, but also about controlling the center and creating a potential pawn storm on d4. This could open up lines for the rook on a1 and potentially create a passed pawn. Of course, the question is, how will black respond to this
### Instruction: You are a professional chess player commenta

In [ ]:
torch.save(model.state_dict(), f'{saved_models_path}final_1e-4_2.bin')

test

In [ ]:
model_path = '/workspace/ChessCommentary/models/final_1e-4_2.bin'

# clear cache
torch.cuda.empty_cache()

In [ ]:
llm_test = LLM()

llm_test_checkpoint = torch.load(model_path)

llm_test.load_model(model_path)
#llm_test.load_state_dict(llm_test_checkpoint['model'])
#llm_test.load_state_dict(llm_test_checkpoint)

llm_test.model = llm_test.model.eval().cuda()

#from transformers import AutoModelForCausalLM

#llm_test = AutoModelForCausalLM.from_pretrained(model_path)
#llm_test.cuda()


Intialization 



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading model



In [ ]:
tested_model = llm_test
max = config['max_length']
eof = '<|endoftext|>'
pad_token_id = tested_model.tokenizer('[PAD]')['input_ids'][0]

In [ ]:
dataloader = test_dataset
def get_results():
    # Set device based on availability of CUDA
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the model to the appropriate device
    tested_model.model.to(device)

    # Retrieve batch data
    processed_data, attn_masks, labels = next(iter(dataloader))

    # Initialize input encodings container
    input_encodings = []

    for i in range(config['batch_size']):
        # Decode the token_ids to text and split at <comment>
        textual_data = tested_model.tokenizer.decode(token_ids=processed_data[i], skip_special_tokens=False).split('<comment>')

        # Target text ends at EOF (assuming EOF is defined elsewhere in your code)
        target_text = textual_data[1].split(eof)[0]
        input_text = textual_data[0]

        # Find the index of <comment> and prepare input encoding
        comment_idx = list(processed_data[i]).index(tested_model.tokenizer.get_added_vocab()['<comment>']) + 1
        input_encoding = processed_data[i][:comment_idx].unsqueeze(0).to(device)

        input_encodings.append(input_encoding)

    # Initialize results container
    results = []

    for input_encoding in input_encodings:
        with torch.no_grad():
            # Generate output using the model's generate function
            outputs = tested_model.model.generate(
                input_encoding,
                num_beams=2,
                no_repeat_ngram_size=2,
                max_length=max + 1,  # assuming 'max' is defined as the maximum length
                pad_token_id=pad_token_id  # assuming 'pad_token_id' is defined elsewhere
            )
            output_text = tested_model.tokenizer.decode(outputs[0], skip_special_tokens=True)
            results.append(output_text)

    return results
